<h1><center>РАБОТА С ПРИЗНАКАМИ</center></h1>

Для начала определите глобальные переменные, которые понадобятся при выполнении заданий. Используйте уже существующий эксперимент в MLflow. Вот пример кода, который можно взять в качестве шаблона для объявления переменных:

In [ ]:
import pandas as pd
import numpy as np
import os
import psycopg
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import ( OneHotEncoder, SplineTransformer, QuantileTransformer, 
                                    RobustScaler, PolynomialFeatures, KBinsDiscretizer )
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from autofeat import AutoFeatRegressor, AutoFeatClassifier

TABLE_NAME = "clean_users_churn" # таблица с данными

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "priznaki" # название эксперимента
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = "priznak_afc" # название зарегистрированной модели 

# experiment_id = mlflow.create_experiment(EXPERIMENT_NAME) 

После этого загрузите ваш набор данных из базы данных Postgres — так, как вы это делали в предыдущих уроках.
<br>Получившийся датафрейм сохраните в переменную df.

In [ ]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {"host": 'rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net', #os.getenv("DB_DESTINATION_HOST"),
                        "port": '6432', #os.getenv("DB_DESTINATION_PORT"),
                        "dbname": 'playground_mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_NAME"),
                        "user": 'mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_USER"),
                        "password": '0c567edd8ad8472e87d5c85cc4d664e4' } #os.getenv("DB_DESTINATION_PASSWORD")}
connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)
df['target'] = (df['end_date'].notna()).astype(int)
df.head(2) 

Выберите команду, которая выделит нечисловые колонки вашего датасета:

In [ ]:
obj_df = df.select_dtypes(include="object")

**Задание 1**

После того как вы выделили категориальные колонки, закодируйте их для подачи в вашу модель:

In [ ]:
# определение категориальных колонок, которые будут преобразованы
cat_columns = ["type", "payment_method", "internet_service", "gender"]

# создание объекта OneHotEncoder для преобразования категориальных переменных
# auto - автоматическое определение категорий
# ignore - игнорировать ошибки, если встречается неизвестная категория
# max_categories - максимальное количество уникальных категорий
# sparse_output - вывод в виде разреженной матрицы, если False, то в виде обычного массива
# drop="first" - удаляет первую категорию, чтобы избежать ловушки мультиколлинеарности
encoder_oh = OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first') # ваш код здесь #

# применение OneHotEncoder к данным. Преобразование категориальных данных в массив
encoded_features = encoder_oh.fit_transform(df[cat_columns].to_numpy()) # ваш код здесь #

# преобразование полученных признаков в DataFrame и установка названий колонок
# get_feature_names_out() - получение имён признаков после преобразования
encoded_df = pd.DataFrame(encoded_features, columns=encoder_oh.get_feature_names_out(cat_columns)) # ваш код здесь #

# конкатенация исходного DataFrame с новым DataFrame, содержащим закодированные категориальные признаки
# axis=1 означает конкатенацию по колонкам
obj_df = pd.concat([obj_df, encoded_df], axis=1)

obj_df.head(2)

Сейчас поработайте с числовыми признаками: monthly_charges и total_charges. Из них можно сгенерировать довольно много признаков для вашей модели. 

**Задание 2**

Напишите код преобразования числовых признаков в списке num_columns, используя следующие энкодеры:
- SplineTransformer,
- QuantileTransformer,
- RobustScaler,
- PolynomialFeatures,
- KBinsDiscretizer.

In [ ]:
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

# num_df = df.select_dtypes(include=['number'])

num_df = df[num_columns].copy()

# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline) # ваш код здесь #
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame( encoded_features, columns=encoder_spl.get_feature_names_out(num_columns) )
num_df = pd.concat([num_df, encoded_df], axis=1)

# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles) #, output_distribution='normal') # ваш код здесь #
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# RobustScaler
encoder_rb = RobustScaler() # ваш код здесь #
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree) # ваш код здесь #
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns)) # ваш код здесь #
# get all columns after the intercept and original features
encoded_df.columns = encoder_pol.get_feature_names_out(num_columns)
encoded_df = encoded_df.iloc[:, 1 + len(num_columns):]
encoded_df.columns = [f"{col}_poly" for col in encoded_df.columns]

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample) # ваш код здесь #
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1) # ваш код здесь #

num_df.head(2)

Применять по отдельности каждое преобразование не всегда удобно, особенно когда их очень много. Поэтому их объединяют в пайплайн или колоночные преобразования, которые можно визуализировать, запустить одной командой и сохранить. Последуйте этому примеру и объедините все энкодеры в одно преобразование. 

**Задание 3**

Напишите код, который объединит преобразования над числовыми колонками в ColumnTransformer, а над категориальными — в Pipeline, используя энкодеры из предыдущих заданий. Затем объедините два получившихся объекта класса одним колоночным преобразованием. После чего объедините ваш преобразованный набор данных с изначальным, а результат сохраните в переменную df. 

In [ ]:
numeric_transformer = ColumnTransformer(
    transformers=[('spl', encoder_spl, num_columns),('q', encoder_q, num_columns), ('rb', encoder_rb, num_columns), ('pol', encoder_pol, num_columns), ('kbd', encoder_kbd, num_columns)] # ваш код здесь #
)

categorical_transformer = Pipeline(
	steps=[('encoder', encoder_oh)] # ваш код здесь #
)

preprocessor = ColumnTransformer(
	transformers=[('num', numeric_transformer, num_columns), ('cat', categorical_transformer, cat_columns)], n_jobs=-1 # ваш код здесь #
)

encoded_features = preprocessor.fit_transform(df) # ваш код здесь #

transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out()) # ваш код здесь #

df = pd.concat([df, transformed_df], axis=1) # ваш код здесь #
df.head(2)

**Задание 4**

Чтобы визуализировать получившееся общее преобразование, просто посмотрите на значение переменной preprocessor.

In [ ]:
preprocessor

Объект preprocessor класса ColumnTransformer, объявленный в предыдущем задании, можно сохранить в MLflow, в директорию preprocessor — точно так же, как и модель. Для этого выполните следующий код в Jupyter Notebook:

In [ ]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = 'https://storage.yandexcloud.net'
os.environ["AWS_ACCESS_KEY_ID"] = "YCAJE3Nlz8iDILW5VTYM1ihQB"
os.environ["AWS_SECRET_ACCESS_KEY"] = "YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44"
os.environ['MLFLOW_ARTIFACT_URI'] = 'http://s3-student-mle-20250529-05fed48463'

TRACKING_SERVER_HOST = '127.0.0.1'
TRACKING_SERVER_PORT = 5000

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.sklearn.log_model(preprocessor, "column_transformer") 

**Задание 5**

Теперь обучите новую модель с новыми признаками, полученными при помощи ваших энкодеров. Посмотрите, как изменилось качество модели на валидационной или тестовой выборке. Зарегистрируйте полученную версию модели.

In [ ]:
df_q = df[['target', 'num__spl__monthly_charges_sp_0',
       'num__spl__monthly_charges_sp_1', 'num__spl__monthly_charges_sp_2',
       'num__spl__monthly_charges_sp_3', 'num__spl__monthly_charges_sp_4',
       'num__spl__monthly_charges_sp_5', 'num__spl__total_charges_sp_0',
       'num__spl__total_charges_sp_1', 'num__spl__total_charges_sp_2',
       'num__spl__total_charges_sp_3', 'num__spl__total_charges_sp_4',
       'num__spl__total_charges_sp_5', 'num__q__monthly_charges',
       'num__q__total_charges', 'num__rb__monthly_charges',
       'num__rb__total_charges', 'num__pol__1', 'num__pol__monthly_charges',
       'num__pol__total_charges', 'num__pol__monthly_charges^2',
       'num__pol__monthly_charges total_charges', 'num__pol__total_charges^2',
       'num__pol__monthly_charges^3',
       'num__pol__monthly_charges^2 total_charges',
       'num__pol__monthly_charges total_charges^2',
       'num__pol__total_charges^3', 'num__kbd__monthly_charges',
       'num__kbd__total_charges', 'cat__type_One year', 'cat__type_Two year',
       'cat__payment_method_Credit card (automatic)',
       'cat__payment_method_Electronic check',
       'cat__payment_method_Mailed check', 'cat__internet_service_Fiber optic',
       'cat__gender_Male']]

# Разделение данных на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(df_q.drop('target', axis=1), df_q['target'], test_size=0.2, random_state=42)

# Инициализация модели
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Обучение модели
model.fit(X_train, y_train)

# Прогнозирование на валидационной выборке
y_pred = model.predict(X_val)

# Оценка качества модели
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy on validation set: {accuracy}")

from mlflow.tracking import MlflowClient
# Регистрация модели с помощью MLFlow
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    mlflow.sklearn.log_model(model, "priznak")
    model_registred_name = "priznak"
    mlflow.register_model("runs:/{}/model".format(run.info.run_id), model_registred_name)
    # model_version_id = mlflow.get_latest_versions(model_registred_name)[0].version
    run_id = run.info.run_id

___

**8/11 Автогенерация признаков**

Вы можете взять вариант разделения набора данных для обучения и валидации, который представлен ниже, или использовать свой вариант.

In [ ]:
features = ['num__spl__monthly_charges_sp_0',
       'num__spl__monthly_charges_sp_1', 'num__spl__monthly_charges_sp_2',
       'num__spl__monthly_charges_sp_3', 'num__spl__monthly_charges_sp_4',
       'num__spl__monthly_charges_sp_5', 'num__spl__total_charges_sp_0',
       'num__spl__total_charges_sp_1', 'num__spl__total_charges_sp_2',
       'num__spl__total_charges_sp_3', 'num__spl__total_charges_sp_4',
       'num__spl__total_charges_sp_5', 'num__q__monthly_charges',
       'num__q__total_charges', 'num__rb__monthly_charges',
       'num__rb__total_charges', 'num__pol__1', 'num__pol__monthly_charges',
       'num__pol__total_charges', 'num__pol__monthly_charges^2',
       'num__pol__monthly_charges total_charges', 'num__pol__total_charges^2',
       'num__pol__monthly_charges^3',
       'num__pol__monthly_charges^2 total_charges',
       'num__pol__monthly_charges total_charges^2',
       'num__pol__total_charges^3', 'num__kbd__monthly_charges',
       'num__kbd__total_charges', 'cat__type_One year', 'cat__type_Two year',
       'cat__payment_method_Credit card (automatic)',
       'cat__payment_method_Electronic check',
       'cat__payment_method_Mailed check', 'cat__internet_service_Fiber optic',
       'cat__gender_Male'] # список признаков вашей модели
target = ['target'] # колонка с таргетом вашей модели

split_column = "begin_date"
test_size = 0.2

# df = df.sort_values(by=[split_column])

X_train, X_test, y_train, y_test = train_test_split( df[features], df[target], test_size=test_size, shuffle=False )

Сгенерируйте новые признаки с помощью библиотеки autofeat. 
<br>Используйте следующие трансформации:
<br>a. обратное деление 1/,
<br>b. подсчёт логарифма,
<br>c. взятие модуля,
<br>d. взятие корня.
<br><br>Для количества шагов, которые необходимо выполнить при создании признаков feateng_steps, установите значение 1.
<br>Для n_jobs установите значение -1 — это означает, что будут использоваться все ядра виртуальной машины.
Набор данных для обучения представлен переменной X_train, запустите подготовленные для него преобразования. Отдельно преобразуйте валидационный или тестовый набор данных X_test. 
Вы можете взять вариант разделения набора данных для обучения и валидации, который представлен ниже, или использовать свой вариант.

In [ ]:
df[ list(df.columns[ df.columns.str.contains('cat_') ]) ] = df[ list(df.columns[ df.columns.str.contains('cat_') ]) ].fillna(0)
df[ list(df.columns[ df.columns.str.contains('num_') ]) ] = df[ list(df.columns[ df.columns.str.contains('num_') ]) ].fillna(0)

In [ ]:
# df = df[  (df.columns[ df.columns.str.contains('cat_|num_') ]) ]

In [ ]:
cat_columns = list(df.columns[ df.columns.str.contains('cat_') ])
num_columns = list(df.columns[ df.columns.str.contains('num_') ])

features = cat_columns + num_columns
transformations = ('1/', 'log', 'abs', 'sqrt')

afc = AutoFeatClassifier(categorical_cols=cat_columns, transformations=transformations, feateng_steps=1, n_jobs=-1)

try:
    X_train_features = afc.fit_transform(X_train, y_train)
    X_test_features = afc.transform(X_test)
except:
    pass

**Задание 2**

Чтобы добиться воспроизводимости, объект afc можно залогировать в MLflow. Делается это так:

In [ ]:
artifact_path = "afc"
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    afc_info = mlflow.sklearn.log_model(afc, artifact_path=artifact_path) 

**Задание 3**

Теперь обучите вашу модель с учётом новых признаков. В конце её нужно зарегистрировать. В переменные ниже вставьте соответствующую информацию об этом.

In [ ]:
df_q = df[['target', 'num__spl__monthly_charges_sp_0',
       'num__spl__monthly_charges_sp_1', 'num__spl__monthly_charges_sp_2',
       'num__spl__monthly_charges_sp_3', 'num__spl__monthly_charges_sp_4',
       'num__spl__monthly_charges_sp_5', 'num__spl__total_charges_sp_0',
       'num__spl__total_charges_sp_1', 'num__spl__total_charges_sp_2',
       'num__spl__total_charges_sp_3', 'num__spl__total_charges_sp_4',
       'num__spl__total_charges_sp_5', 'num__q__monthly_charges',
       'num__q__total_charges', 'num__rb__monthly_charges',
       'num__rb__total_charges', 'num__pol__1', 'num__pol__monthly_charges',
       'num__pol__total_charges', 'num__pol__monthly_charges^2',
       'num__pol__monthly_charges total_charges', 'num__pol__total_charges^2',
       'num__pol__monthly_charges^3',
       'num__pol__monthly_charges^2 total_charges',
       'num__pol__monthly_charges total_charges^2',
       'num__pol__total_charges^3', 'num__kbd__monthly_charges',
       'num__kbd__total_charges', 'cat__type_One year', 'cat__type_Two year',
       'cat__payment_method_Credit card (automatic)',
       'cat__payment_method_Electronic check',
       'cat__payment_method_Mailed check', 'cat__internet_service_Fiber optic',
       'cat__gender_Male']]

# Разделение данных на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(df_q.drop('target', axis=1), df_q['target'], test_size=0.2, random_state=42)

# Инициализация модели
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Обучение модели
model.fit(X_train, y_train)

# Прогнозирование на валидационной выборке
y_pred = model.predict(X_val)

# Оценка качества модели
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy on validation set: {accuracy}")

from mlflow.tracking import MlflowClient
# Регистрация модели с помощью MLFlow
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    mlflow.sklearn.log_model(model, "priznak_afc")
    model_registred_name = "priznak_afc"
    mlflow.register_model("runs:/{}/model".format(run.info.run_id), model_registred_name)
    # model_version_id = mlflow.get_latest_versions(model_registred_name)[0].version
    run_id = run.info.run_id